# **Transportation Problem:**

**Problem Statement:**

You are managing the distribution of goods from multiple suppliers to multiple consumers. Each supplier has a certain quantity of goods available for distribution, and each consumer has a certain demand for these goods. The goal is to determine the most cost-effective way to transport the goods from suppliers to consumers while minimizing transportation costs.

Details:

There are two suppliers: $S_1$ and $S_2$.

There are three consumers: $C_1$, $C_2$, and $C_3$.

The cost of shipping one unit of goods from each supplier to each consumer is given as follows:

From $S_1$ to $C_1$: $3 per unit

From $S_1$ to $C_2$: $2 per unit

From $S_1$ to $C_3$: $7 per unit

From $S_2$ to $C_1$: $2 per unit

From $S_2$ to $C_2$: $5 per unit

From $S_2$ to $C_3$: $1 per unit

The supply available from each supplier is as follows:

$S_1$ has $10$ units available.
$S_2$ has $20$ units available.

The demand from each consumer is as follows:

$C_1$ requires $15$ units.

$C_2$ requires $10$ units.

$C_3$ requires $5$ units.

Your task is to determine how many units to ship from each supplier to each consumer to meet the demand while minimizing the total transportation cost. Use linear programming to find the optimal shipment quantities.







**Modeling:**



In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

In [2]:
#Defining the problem

costs = {("S1", "C1"): 3, ("S1", "C2"): 2, ("S1", "C3"): 7, ("S2", "C1"): 2, ("S2", "C2"): 5, ("S2", "C3"): 1}
supply = {"S1": 10, "S2": 20}
demand = {"C1": 15, "C2": 10, "C3": 5}

In [3]:
#Defining variables

x = {}
for supplier in ["S1", "S2"]:
    for consumer in ["C1", "C2", "C3"]:
        x[(supplier, consumer)] = pulp.LpVariable(f"shipment_{supplier}_{consumer}", lowBound=0, cat="Continuous")

In [4]:
#Defining Objective Function

Model = LpProblem("Transportation_Problem", LpMinimize)
Model += sum(x[(supplier, consumer)] * costs[(supplier, consumer)] for supplier in ["S1", "S2"] for consumer in ["C1", "C2", "C3"])

In [5]:
#Constraints

#Define supply constraints
for supplier in ["S1", "S2"]:
    Model += sum(x[(supplier, consumer)] for consumer in ["C1", "C2", "C3"]) == supply[supplier]

#Define demand constraints
for consumer in ["C1", "C2", "C3"]:
    Model += sum(x[(supplier, consumer)] for supplier in ["S1", "S2"]) == demand[consumer]

In [6]:
#Model summary

Model

Transportation_Problem:
MINIMIZE
3*shipment_S1_C1 + 2*shipment_S1_C2 + 7*shipment_S1_C3 + 2*shipment_S2_C1 + 5*shipment_S2_C2 + 1*shipment_S2_C3 + 0
SUBJECT TO
_C1: shipment_S1_C1 + shipment_S1_C2 + shipment_S1_C3 = 10

_C2: shipment_S2_C1 + shipment_S2_C2 + shipment_S2_C3 = 20

_C3: shipment_S1_C1 + shipment_S2_C1 = 15

_C4: shipment_S1_C2 + shipment_S2_C2 = 10

_C5: shipment_S1_C3 + shipment_S2_C3 = 5

VARIABLES
shipment_S1_C1 Continuous
shipment_S1_C2 Continuous
shipment_S1_C3 Continuous
shipment_S2_C1 Continuous
shipment_S2_C2 Continuous
shipment_S2_C3 Continuous

In [7]:
#Solving model

Model.solve()
print('Optimal Solution:', pulp.value(Model.objective))

Optimal Solution: 55.0


In [8]:
#Optimal values of variables

for variables in Model.variables():
  print('{}:{}'.format(variables.name, variables.varValue))

shipment_S1_C1:0.0
shipment_S1_C2:10.0
shipment_S1_C3:0.0
shipment_S2_C1:15.0
shipment_S2_C2:0.0
shipment_S2_C3:5.0


In [9]:
VNames = []
for variables in Model.variables():
  VNames.append(variables.name)

VValue = []
for variables in Model.variables():
  VValue.append(int(variables.varValue))

data = {'Variables': VNames, 'Value': VValue}
pd.DataFrame(data, index = range(1, len(VValue)+1))

,Variables,Value
1,shipment_S1_C1,0
2,shipment_S1_C2,10
3,shipment_S1_C3,0
4,shipment_S2_C1,15
5,shipment_S2_C2,0
6,shipment_S2_C3,5


In [10]:
print('Current Status: ', LpStatus[Model.status])

Current Status:  Optimal
